train test 데이터에서  500개 줄여서 테스트

In [38]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

import pandas as pd 
import os
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten
from keras.utils import to_categorical

from keras.models import Sequential,load_model

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
np.random.seed(1)

In [63]:
train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split = 0.2,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.7,
    rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './images2/train',
        target_size = (300,300),
        batch_size=50,
        #color_mode ='rgb',
        classes = ['food','interior', 'exterior'],
        shuffle =True,
        seed = 42,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        './images2/test',
        classes = ['food','interior', 'exterior'],
        target_size=(300,300),
        batch_size=10,
#         color_mode ='rgb',
        shuffle =True,
        seed = 42,
        class_mode='categorical')

Found 36000 images belonging to 3 classes.
Found 9000 images belonging to 3 classes.


In [64]:
model = Sequential()
model.add(Conv2D(32, kernel_size=3,
                 activation='selu',
                 input_shape=(300,300,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(16, 3, activation='selu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
# model.add(Dense(8, activation='selu'))
# model.add(Dense(16, activation='relu'))
model.add(Dense(3, activation='softmax'))

# 3. 모델 학습과정 설정 및 요약
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 모델 학습시키기
model.fit_generator(
    train_generator,
    steps_per_epoch=10,
    epochs=10,
    validation_data=test_generator,
    validation_steps=5)

model.save('model-13')

# 5. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 6. 모델 사용하기
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 298, 298, 32)      896       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 147, 147, 16)      4624      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 73, 73, 16)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 85264)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 255795    
Total params: 261,315
Trainable params: 261,315
Non-trainable params: 0
_______________________________________________

In [65]:
print(train_generator.class_indices)
print(test_generator.class_indices)

{'food': 0, 'interior': 1, 'exterior': 2}
{'food': 0, 'interior': 1, 'exterior': 2}
